In [1]:
from PIL import Image
import imagehash
from image_similarity import check_none_hash_color
hash = imagehash.average_hash(Image.open('training/output_0242.jpg'))
other_hash = imagehash.average_hash(Image.open('training/output_0250.jpg'))
print(hash,'\n',other_hash)

fff3f1f1f9d3c1e1 
 fff3f1f1f1d3c3e1


## What hashing?
I used average hashing because I found it gave the best results. 

## Data Preparation
Converting each frame in badapple into an average_hash. Also map which hashes are to which frames.

In [ ]:
import math

TRAINING_DATA_NUM = 6572 # 6572 training images
TRAINING_DATA_PREFIX = 'training/output_'

hash_indices = {"0000000000000000" : [], "ffffffffffffffff" : []}

for i in range(1, TRAINING_DATA_NUM+1):
    file_number = '0' * (3 - int(math.log10(i))) + str(i)
    path = f"{TRAINING_DATA_PREFIX}{file_number}.jpg"

    image_file = Image.open(path)
    gray = image_file.convert('L')
    threshold = image_file.point( lambda p: 255 if p > 127 else 0 )
    mono = image_file.convert('1') 
    hash = str(imagehash.dhash(mono))
    if hash == "0000000000000000": # add color cluster, but only for full black/white. average hash can't differentiate colors.
        if check_none_hash_color(threshold): # if black
            hash_indices["ffffffffffffffff"].append(i)
        else:
            hash_indices["0000000000000000"].append(i)
    elif hash not in hash_indices:
        hash_indices[hash] = [i]
    else:
        hash_indices[hash].append(i)

In [ ]:
# Convert it to json
import json

json_dict = {"Hash Information": hash_indices}
with open("hash_indices.json", 'w') as f:
    json.dump(json_dict, f, indent=4, sort_keys=True)